In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f


spark = SparkSession.builder \
      .appName("data_preprocess") \
      .getOrCreate()

23/05/25 19:46:51 WARN Utils: Your hostname, Shihs-PC.local resolves to a loopback address: 127.0.0.1; using 172.18.170.52 instead (on interface en0)
23/05/25 19:46:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/25 19:46:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read data

In [3]:
traffic_df = spark.read.parquet("data_parquet/*/*", header = True)
traffic_df.show(5)

23/05/25 19:46:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+--------+------------+------------------+----------------------------------+--------+------------+-----------------------+--------------+------------------+------------------+------------------+------------------------+---------------------+---------------------+---------------------+-------------------+---------------------+-----------------+-----------------+-----------------+-------------------------------+-------------------------------+--------------------------------------------+------------------------------------+----------------------------------+------------------------+------------------------+-----------------------+----------------------------+------------+----------+-------------------------------+-------------------------------+------+----+--------------------+----------------------------------+------------------------------------+------------------------+------------------------+---------------------------+---------------------------+--------------------------

### Combine with calendar

In [4]:
filename = "108-111calendars"
calendar = spark.read.csv(filename, header=True, encoding='utf8')

calendar = calendar.drop(calendar.columns[1],calendar.columns[3]) \
              .withColumnRenamed(calendar.columns[0],"Date") \
              .withColumnRenamed(calendar.columns[2],"holiday")

df = traffic_df.join(calendar, traffic_df.發生日期 ==  calendar.Date, "inner") \
      .drop(calendar.Date)
df.show(5)

+--------+--------+------------+------------------+----------------------------------+--------+------------+-----------------------+--------------+------------------+------------------+------------------+------------------------+---------------------+---------------------+---------------------+-------------------+---------------------+-----------------+-----------------+-----------------+-------------------------------+-------------------------------+--------------------------------------------+------------------------------------+----------------------------------+------------------------+------------------------+-----------------------+----------------------------+------------+----------+-------------------------------+-------------------------------+------+----+--------------------+----------------------------------+------------------------------------+------------------------+------------------------+---------------------------+---------------------------+--------------------------

In [10]:
df.printSchema()

root
 |-- 發生日期: string (nullable = true)
 |-- 發生時間: string (nullable = true)
 |-- 事故類別名稱: string (nullable = true)
 |-- 處理單位名稱警局層: string (nullable = true)
 |-- 發生地點: string (nullable = true)
 |-- 天候名稱: string (nullable = true)
 |-- 光線名稱: string (nullable = true)
 |-- 道路類別-第1當事者-名稱: string (nullable = true)
 |-- 速限-第1當事者: string (nullable = true)
 |-- 道路型態大類別名稱: string (nullable = true)
 |-- 道路型態子類別名稱: string (nullable = true)
 |-- 事故位置大類別名稱: string (nullable = true)
 |-- 事故位置子類別名稱: string (nullable = true)
 |-- 路面狀況-路面鋪裝名稱: string (nullable = true)
 |-- 路面狀況-路面狀態名稱: string (nullable = true)
 |-- 路面狀況-路面缺陷名稱: string (nullable = true)
 |-- 道路障礙-障礙物名稱: string (nullable = true)
 |-- 道路障礙-視距品質名稱: string (nullable = true)
 |-- 道路障礙-視距名稱: string (nullable = true)
 |-- 號誌-號誌種類名稱: string (nullable = true)
 |-- 號誌-號誌動作名稱: string (nullable = true)
 |-- 車道劃分設施-分向設施大類別名稱: string (nullable = true)
 |-- 車道劃分設施-分向設施子類別名稱: string (nullable = true)
 |-- 車道劃分設施-分道設施-快車道或一般車道間名稱: string (nullable = true)

23/05/25 20:14:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 131223 ms exceeds timeout 120000 ms
23/05/25 20:14:36 WARN SparkContext: Killing executors is not supported by current scheduler.


### Missing Value

In [64]:
import pyspark.sql.functions as f
null_count = df.select([ f.count(f.when( df[c].isNull(), "" )).alias(c) for c in df.columns]).collect()

for c in range(len(df.columns)):
    if null_count[0][c] != 0:
        print(df.columns[c], )

當事者區分-類別-大類別名稱-車種
當事者區分-類別-子類別名稱-車種
保護裝備名稱
行動電話或電腦或其他相類功能裝置名稱
當事者行動狀態大類別名稱
當事者行動狀態子類別名稱
車輛撞擊部位大類別名稱-最初
車輛撞擊部位子類別名稱-最初
車輛撞擊部位大類別名稱-其他
車輛撞擊部位子類別名稱-其他
肇因研判大類別名稱-個別
肇因研判子類別名稱-個別
肇事逃逸類別名稱-是否肇逃
